In [174]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import sys
import matplotlib.pyplot as plt
from IPython.display import Image
import cv2
import os
import glob

In [175]:
def goods(name):
    
    my_variable = name

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print('video read failed')
        sys.exit()

    num = 0
    
    os.mkdir(f'{name}')
    
    while True:

        ret , frame = cap.read()

        if not ret:
            print('Vodeo read failed')
            break

        cv2.imshow('frame',frame)

        key = cv2.waitKey(20)

        if key == 27:
            break

        elif key == ord('s'):
            frame_filename = f"./{name}/{num:04d}.png"
            cv2.imwrite(frame_filename, frame)
            target.append(i)
            num += 1
            
    cap.release()
    cv2.destroyAllWindows()
    
    return my_variable

In [192]:
names = []
target = []

cnt = int(input())

for i in range(cnt):
    name = goods(input(''))
    names.append(name)

1
1


In [178]:
img_paths = []

for i in range(len(names)):
    img_path = glob.glob(f'./{names[i]}/*.*')
    img_paths.append(img_path)
img_paths = [y for x in img_paths for y in x]


In [179]:
imgs = np.zeros((0, 150, 150, 3), np.float32)

for img_path in img_paths:
    img = cv2.imread(img_path)
    img = cv2.resize(img, (150, 150))
    img = (img.astype(np.float32))/255.
    img = np.expand_dims(img, axis=0)
    imgs = np.append(imgs, img, axis=0)
    

In [180]:
target = np.array(target)
print(imgs.shape, target.shape)
print(len(imgs), len(target))

(115, 150, 150, 3) (115,)
115 115


In [181]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
imgs, target, test_size = 0.2, shuffle=True)

In [186]:
print(X_train.shape, y_train.shape, y_train[4])

cv2.imshow('img', X_train[4])
cv2.waitKey()
cv2.destroyAllWindows()

(92, 150, 150, 3) (92,) 0


In [187]:
transfer_vgg16 = keras.applications.vgg16.VGG16(
    weights = 'imagenet',
    include_top = False,
input_shape = (150,150,3))

transfer_vgg16.trainable = False
transfer_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [188]:
finetune_vgg16 = keras.Sequential()
finetune_vgg16.add(transfer_vgg16)
finetune_vgg16.add(keras.layers.Flatten())
finetune_vgg16.add(keras.layers.Dense(64,activation='relu'))
finetune_vgg16.add(keras.layers.Dropout(0.3))
finetune_vgg16.add(keras.layers.Dense(1, activation='softmax'))

finetune_vgg16.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_20 (Flatten)        (None, 8192)              0         
                                                                 
 dense_55 (Dense)            (None, 64)                524352    
                                                                 
 dropout_20 (Dropout)        (None, 64)                0         
                                                                 
 dense_56 (Dense)            (None, 1)                 65        
                                                                 
Total params: 15,239,105
Trainable params: 524,417
Non-trainable params: 14,714,688
_________________________________________________________________


In [189]:
finetune_vgg16.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['acc'])

In [190]:
history = finetune_vgg16.fit(X_train, y_train,
                            epochs=10,
                            batch_size=20,
                            validation_data=(X_val, y_val))

Epoch 1/10


C:\Users\jinoo\anaconda3\envs\project\lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


5/5 [==============================] - 12s 2s/step - loss: 0.0000e+00 - acc: 0.3152 - val_loss: 0.0000e+00 - val_acc: 0.2609
Epoch 2/10
5/5 [==============================] - 10s 2s/step - loss: 0.0000e+00 - acc: 0.3152 - val_loss: 0.0000e+00 - val_acc: 0.2609
Epoch 3/10
5/5 [==============================] - 10s 2s/step - loss: 0.0000e+00 - acc: 0.3152 - val_loss: 0.0000e+00 - val_acc: 0.2609
Epoch 4/10
5/5 [==============================] - 11s 2s/step - loss: 0.0000e+00 - acc: 0.3152 - val_loss: 0.0000e+00 - val_acc: 0.2609
Epoch 5/10
3/5 [=================>............] - ETA: 3s - loss: 0.0000e+00 - acc: 0.2833

KeyboardInterrupt: 